In [1]:
import torch
import math
import torch_frame
import copy
from tqdm import tqdm
from utils.data import DatabaseFactory
from utils.builder import build_pyg_hetero_graph
from utils.resource import get_text_embedder_cfg
from utils.util import load_col_types
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# cache_dir = "./data/rel-trial-tensor-frame"

/home/lingze/anaconda3/envs/deepdb/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# db = DatabaseFactory.get_db("trial", with_text_compress=True)
# task = DatabaseFactory.get_task("trial", "study-outcome")
# cache_dir = "./data/rel-trial-tensor-frame"
# db_name = "trial"

# db = DatabaseFactory.get_db("avito", with_text_compress=True)
# task = DatabaseFactory.get_task("avito", "ad-ctr")
# cache_dir = "./data/rel-avito-tensor-frame"
# db_name = "avito"
# db = DatabaseFactory.get_db("stack", cache_dir="/home/lingze/.cache/relbench/stack" , with_text_compress=True)
# task = DatabaseFactory.get_task("stack", "user-engagement")
# cache_dir = "./data/stack-tensor-frame"
# db_name = "stack"


# db = DatabaseFactory.get_db("event", with_text_compress=True)
# task = DatabaseFactory.get_task("event", "user-ignore")
# cache_dir = "./data/rel-event-tensor-frame"
# db_name = "event"
# # edge_path = "./edges/rel-event-edges.npz"
# sample_path = "./samples/rel-event-samples.npz"
# edge_path = None

db_cache_dir = "/home/lingze/.cache/relbench/ratebeer"
db = DatabaseFactory.get_db("ratebeer", cache_dir=db_cache_dir)
cache_dir = "./data/ratebeer-tensor-frame"
db_name = "ratebeer"
edge_path = "./edges/ratebeer-edges.npz"
sample_path = "./samples/ratebeer-samples.npz"

Loading Database object from /home/lingze/.cache/relbench/ratebeer/db...
Done in 0.61 seconds.


In [3]:
col_type_dict = load_col_types(cache_path=cache_dir, file_name = "col_type_dict.pkl")

In [4]:
data, col_stats_dict = build_pyg_hetero_graph(
    db,
    col_type_dict,
    get_text_embedder_cfg(device="cpu"),
    cache_dir=cache_dir,
    verbose=True,
)

/home/lingze/anaconda3/envs/deepdb/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


-----> Materialize favorites Tensor Frame
-----> Materialize beers Tensor Frame
-----> Materialize places Tensor Frame
-----> Materialize availability Tensor Frame
-----> Materialize place_ratings Tensor Frame
-----> Materialize beer_ratings Tensor Frame
-----> Materialize countries Tensor Frame
-----> Materialize brewers Tensor Frame
-----> Materialize users Tensor Frame


In [7]:
# add new edges:
from utils.util import load_np_dict
from torch_geometric.utils import sort_edge_index

if edge_path is not None:
    edge_dict = load_np_dict(edge_path)

    for edge_name, edge_np in edge_dict.items():
        print(f"Adding edge {edge_name} with shape {edge_np.shape}")
        src_table, dst_table = edge_name.split('-')[0], edge_name.split('-')[1]
        edge_index = torch.from_numpy(edge_np.astype(int)).t()
        # [2, edge_num]
        edge_type = (src_table, f"appendix", dst_table)
        data[edge_type].edge_index = sort_edge_index(edge_index)
    data.validate()

Adding edge places-brewers with shape (8408, 2)
Adding edge places-users with shape (6187, 2)
Adding edge places-beers with shape (7826, 2)
Adding edge countries-users with shape (83, 2)
Adding edge countries-beers with shape (71, 2)
Adding edge brewers-places with shape (8144, 2)
Adding edge brewers-users with shape (10119, 2)
Adding edge users-places with shape (26109, 2)
Adding edge users-countries with shape (16350, 2)
Adding edge users-brewers with shape (18127, 2)
Adding edge users-beers with shape (20677, 2)
Adding edge beers-places with shape (35062, 2)
Adding edge beers-countries with shape (25480, 2)
Adding edge beers-users with shape (29988, 2)


In [8]:
import numpy as np
import pandas as pd
from relbench.base import Database
from torch_geometric.data import HeteroData
from torch_geometric.loader import NeighborLoader
from typing import List
from utils.util import to_unix_time


def neighborsample_loader(
    data: HeteroData,
    db: Database,
    entity_table: str,
    batch_size: int = 256,
    num_neighbors: List[int] = [64, 64]
):

    n = len(db.table_dict[entity_table].df)
    node_idxs = np.arange(n)
    nodes = (entity_table, torch.from_numpy(node_idxs))
    
    input_time = torch.from_numpy(
        to_unix_time(pd.Series([db.max_timestamp] * n)))

    if db.table_dict[entity_table].time_col:
        time_col = db.table_dict[entity_table].time_col
        time_values = db.table_dict[entity_table].df[time_col].loc[node_idxs.tolist(
        )]
        input_time = torch.from_numpy(to_unix_time(time_values))

    loader = NeighborLoader(
        data,
        num_neighbors=num_neighbors,
        input_nodes=nodes,
        time_attr="time",
        input_time=input_time,
        batch_size=batch_size,
        temporal_strategy="uniform",
        disjoint=True,
        shuffle=True,
        num_workers=0,
    )
    return loader

In [9]:
def neighborsample_batch(
    data: HeteroData,
    db: Database,
    entity_table: str,
    node_idxs: np.ndarray,
    num_neighbors: List[int] = [64,64],
):
    # node_idxs: [n]
    nodes = (entity_table, torch.from_numpy(node_idxs))
    n = node_idxs.shape[0]
    input_time = torch.from_numpy(
        to_unix_time(pd.Series([db.max_timestamp] * n)))

    if db.table_dict[entity_table].time_col:
        time_col = db.table_dict[entity_table].time_col
        time_values = db.table_dict[entity_table].df[time_col].loc[node_idxs.tolist(
        )]
        input_time = torch.from_numpy(to_unix_time(time_values))

    loader = NeighborLoader(
        data,
        num_neighbors=num_neighbors,
        input_nodes=nodes,
        time_attr = "time",
        input_time=input_time,
        batch_size=n,
        temporal_strategy="uniform",
        shuffle=False,
        disjoint=True,
        num_workers=0,
        persistent_workers=False,
    )
    return next(iter(loader))

In [10]:
from utils.util import load_np_dict
sample_dict = load_np_dict(sample_path)

entity_tables = list(sample_dict.keys())

In [11]:
# declare the model
from model.rdb import RDBModel
channels = 128
out_channels = 1

net = RDBModel(
    data,
    col_stats_dict,
    channels=channels,
    out_channels=out_channels,
    feat_layer_num=1,
    feat_norm="layer_norm",
    head_norm="batch_norm",
    aggr="sum",
    graph_layer_num=2,
    dropout_prob=0.3
)

In [12]:
from model.utils import InfoNCE
import time
import random

lr = 0.0001
negative_sample_pool_size = 512
temperature = 0.01
optimizer = torch.optim.Adam(net.parameters(), lr = lr)
epoches = 300
batch_size = 1024
max_steps_in_epoch = 10
negative_num = 20
loss_fn = InfoNCE(temperature=temperature, negative_mode = 'paired', reduction='mean')

In [13]:
# expand sample_dict
positive_pair_index_dict = {}
# entity_table -> np.ndarray (positive pair matrix)
for table_name in entity_tables:
    sample_np = sample_dict[table_name]
    sample_idx = sample_np[:, 0]
    sample_pos_pair = sample_np
    n = db.table_dict[table_name].df.shape[0]
    b = sample_np.shape[1]
    arr = np.full((n,b), -1)
    arr[:, 0] = np.arange(n)
    arr[sample_idx,:] = sample_pos_pair
    positive_pair_index_dict[table_name] = arr

In [14]:
loader_dict = {}
for table_name in entity_tables:
    loader_dict[table_name] = neighborsample_loader(
        data,
        db,
        entity_table=table_name,
        batch_size=batch_size,
        num_neighbors=[64, 64],
    )

In [15]:
from model.augment import HeteroGraphPermutationTool
permutation_drop_prob = 0.2
net.to(device)
best_loss = math.inf
best_state = None
patience = 0
early_stop_threshold = 1
net.train()

for epoch in range(1, epoches + 1):
    ave_loss = 0
    random.shuffle(entity_tables)

    for sample_table in entity_tables:
        positive_pair_index = positive_pair_index_dict[sample_table]
        loader = loader_dict[sample_table]

        loss_accum = count_accum = 0
        now = time.time()

        cnt = 0
        for batch in tqdm(loader,
                          leave=False,
                          desc=f"Epoch {epoch}:{sample_table}"):
            cnt += 1
            if cnt > max_steps_in_epoch:
                break

            optimizer.zero_grad()

            # get the positive pair batch
            input_node_idx = batch[sample_table].input_id
            B = input_node_idx.shape[0]
            batch_positive_pair_index = positive_pair_index[input_node_idx]
            batch_pos_samples_nodes = np.array(
                [np.random.choice(row[row != -1]) for row in batch_positive_pair_index])

            pos_batch = neighborsample_batch(
                data, db, sample_table, batch_pos_samples_nodes)

            batch, pos_batch = batch.to(device), pos_batch.to(device)

            permute_pos_batch = HeteroGraphPermutationTool.random_permutation(
                pos_batch,
                drop_prob=permutation_drop_prob,
                exclude_node_types=[sample_table]
            )

            anchor_x = net.get_node_embedding(batch, sample_table)[
                sample_table][:B]
            permute_x_dict = net.get_node_embedding(
                permute_pos_batch, sample_table)
            pos_x = permute_x_dict[sample_table][:B]
            # [B, channels]

            # construct the negative samples
            neg_indices = torch.randint(
                0, len(permute_x_dict[sample_table]), (B, negative_num))
            neg_x = permute_x_dict[sample_table][neg_indices]
            # [B, negative_num, channels]

            loss = loss_fn(anchor_x, pos_x, neg_x)
            loss.backward()
            optimizer.step()

            loss_accum += loss.detach().item() * B
            count_accum += B

        end = time.time()
        train_loss = loss_accum / count_accum
        ave_loss += train_loss
        mins, secs = divmod(end - now, 60)
        print(
            f"====> {epoch}  In {sample_table}, Train loss: {train_loss} Count accum :{count_accum}, Cost Time {mins:.0f}m {secs:.0f}s ")

    ave_loss /= len(entity_tables)
    if ave_loss < best_loss:
        best_loss = ave_loss
        best_state = copy.deepcopy(net.state_dict())
        print(f"Best loss: {best_loss}")
        patience = 0
    else:
        patience += 1
        if patience >= early_stop_threshold:
            print(f"Early stopping at epoch {epoch}")
            break
        print(
            f"Patience: {patience}/{early_stop_threshold}, Best loss: {best_loss}")

====> 1  In places, Train loss: 4.341405510902405 Count accum :10240, Cost Time 0m 23s 


====> 1  In brewers, Train loss: 4.219197130203247 Count accum :10240, Cost Time 0m 20s 


====> 1  In users, Train loss: 12.680357074737548 Count accum :10240, Cost Time 0m 18s 


====> 1  In countries, Train loss: 4.600297451019287 Count accum :251, Cost Time 0m 2s 


====> 1  In beers, Train loss: 8.370619821548463 Count accum :10240, Cost Time 0m 19s 
Best loss: 6.84237539768219


...

====> 28  In countries, Train loss: 1.5229995250701904 Count accum :251, Cost Time 0m 2s 


====> 28  In beers, Train loss: 0.3698554515838623 Count accum :10240, Cost Time 0m 15s 


====> 28  In brewers, Train loss: 0.21372863203287124 Count accum :10240, Cost Time 0m 17s 


====> 28  In places, Train loss: 0.2442854732275009 Count accum :10240, Cost Time 0m 18s 


====> 28  In users, Train loss: 1.2023052334785462 Count accum :10240, Cost Time 0m 15s 
Early stopping at epoch 28


In [16]:
import torch
import json
torch.save(best_state, "./static/method/ratebeer-pre-trained.pth")
# torch.save(best_state, "./static/method/rel-event-pre-trained-wo-edges.pth")